# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 8:30AM,241699,10,SONSB0001864,"Nextsource Biotechnology, LLC",Released
1,Jul 13 2022 8:30AM,241699,10,SONSB0001865,"Nextsource Biotechnology, LLC",Released
2,Jul 13 2022 8:30AM,241699,10,SONSB0001866,"Nextsource Biotechnology, LLC",Released
3,Jul 13 2022 8:30AM,241699,10,SONSB0001867,"Nextsource Biotechnology, LLC",Released
4,Jul 13 2022 8:30AM,241699,10,SOTAH0000726,"Nextsource Biotechnology, LLC",Released
5,Jul 13 2022 8:30AM,241698,10,8266296,"Citieffe, Inc.",Released
6,Jul 12 2022 3:54PM,241606,15,8260688,"Carwin Pharmaceutical Associates, LLC",Released
7,Jul 12 2022 3:14PM,241697,12,8266285,Doctors Scientific Organica,Released
8,Jul 12 2022 3:14PM,241696,10,8266281,"ClearSpec, LLC",Released
9,Jul 12 2022 3:10PM,241695,21,620263,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,241695,Released,1
38,241696,Released,1
39,241697,Released,1
40,241698,Released,1
41,241699,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241695,NaN,NaN,1.0
241696,NaN,NaN,1.0
241697,NaN,NaN,1.0
241698,NaN,NaN,1.0
241699,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241605,27.0,30.0,0.0
241606,0.0,0.0,1.0
241612,0.0,0.0,1.0
241614,0.0,1.0,0.0
241623,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241605,27,30,0
241606,0,0,1
241612,0,0,1
241614,0,1,0
241623,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241605,27,30,0
1,241606,0,0,1
2,241612,0,0,1
3,241614,0,1,0
4,241623,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241605,27,30,
1,241606,,,1
2,241612,,,1
3,241614,,1,
4,241623,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 8:30AM,241699,10,"Nextsource Biotechnology, LLC"
5,Jul 13 2022 8:30AM,241698,10,"Citieffe, Inc."
6,Jul 12 2022 3:54PM,241606,15,"Carwin Pharmaceutical Associates, LLC"
7,Jul 12 2022 3:14PM,241697,12,Doctors Scientific Organica
8,Jul 12 2022 3:14PM,241696,10,"ClearSpec, LLC"
9,Jul 12 2022 3:10PM,241695,21,"NBTY Global, Inc."
10,Jul 12 2022 3:07PM,241694,20,"ACI Healthcare USA, Inc."
11,Jul 12 2022 3:05PM,241693,10,"ClearSpec, LLC"
12,Jul 12 2022 3:02PM,241692,10,Yusen – 3D Matrix
13,Jul 12 2022 2:51PM,241690,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 13 2022 8:30AM,241699,10,"Nextsource Biotechnology, LLC",,,5
1,Jul 13 2022 8:30AM,241698,10,"Citieffe, Inc.",,,1
2,Jul 12 2022 3:54PM,241606,15,"Carwin Pharmaceutical Associates, LLC",,,1
3,Jul 12 2022 3:14PM,241697,12,Doctors Scientific Organica,,,1
4,Jul 12 2022 3:14PM,241696,10,"ClearSpec, LLC",,,1
5,Jul 12 2022 3:10PM,241695,21,"NBTY Global, Inc.",,,1
6,Jul 12 2022 3:07PM,241694,20,"ACI Healthcare USA, Inc.",,,1
7,Jul 12 2022 3:05PM,241693,10,"ClearSpec, LLC",,,1
8,Jul 12 2022 3:02PM,241692,10,Yusen – 3D Matrix,,1,
9,Jul 12 2022 2:51PM,241690,10,"Nextsource Biotechnology, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 8:30AM,241699,10,"Nextsource Biotechnology, LLC",5,,
1,Jul 13 2022 8:30AM,241698,10,"Citieffe, Inc.",1,,
2,Jul 12 2022 3:54PM,241606,15,"Carwin Pharmaceutical Associates, LLC",1,,
3,Jul 12 2022 3:14PM,241697,12,Doctors Scientific Organica,1,,
4,Jul 12 2022 3:14PM,241696,10,"ClearSpec, LLC",1,,
5,Jul 12 2022 3:10PM,241695,21,"NBTY Global, Inc.",1,,
6,Jul 12 2022 3:07PM,241694,20,"ACI Healthcare USA, Inc.",1,,
7,Jul 12 2022 3:05PM,241693,10,"ClearSpec, LLC",1,,
8,Jul 12 2022 3:02PM,241692,10,Yusen – 3D Matrix,,1,
9,Jul 12 2022 2:51PM,241690,10,"Nextsource Biotechnology, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 8:30AM,241699,10,"Nextsource Biotechnology, LLC",5,,
1,Jul 13 2022 8:30AM,241698,10,"Citieffe, Inc.",1,,
2,Jul 12 2022 3:54PM,241606,15,"Carwin Pharmaceutical Associates, LLC",1,,
3,Jul 12 2022 3:14PM,241697,12,Doctors Scientific Organica,1,,
4,Jul 12 2022 3:14PM,241696,10,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 8:30AM,241699,10,"Nextsource Biotechnology, LLC",5.0,NaN,NaN
1,Jul 13 2022 8:30AM,241698,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Jul 12 2022 3:54PM,241606,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN
3,Jul 12 2022 3:14PM,241697,12,Doctors Scientific Organica,1.0,NaN,NaN
4,Jul 12 2022 3:14PM,241696,10,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 8:30AM,241699,10,"Nextsource Biotechnology, LLC",5.0,0.0,0.0
1,Jul 13 2022 8:30AM,241698,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Jul 12 2022 3:54PM,241606,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0
3,Jul 12 2022 3:14PM,241697,12,Doctors Scientific Organica,1.0,0.0,0.0
4,Jul 12 2022 3:14PM,241696,10,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5075076,121.0,4.0,0.0
12,241697,1.0,0.0,0.0
15,483211,1.0,30.0,27.0
18,241669,1.0,0.0,0.0
19,483308,1.0,1.0,0.0
20,1208407,45.0,10.0,2.0
21,966594,3.0,1.0,0.0
22,483297,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5075076,121.0,4.0,0.0
1,12,241697,1.0,0.0,0.0
2,15,483211,1.0,30.0,27.0
3,18,241669,1.0,0.0,0.0
4,19,483308,1.0,1.0,0.0
5,20,1208407,45.0,10.0,2.0
6,21,966594,3.0,1.0,0.0
7,22,483297,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,121.0,4.0,0.0
1,12,1.0,0.0,0.0
2,15,1.0,30.0,27.0
3,18,1.0,0.0,0.0
4,19,1.0,1.0,0.0
5,20,45.0,10.0,2.0
6,21,3.0,1.0,0.0
7,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,121.0
1,12,Released,1.0
2,15,Released,1.0
3,18,Released,1.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,27.0,0.0,0.0,2.0,0.0,0.0
Executing,4.0,0.0,30.0,0.0,1.0,10.0,1.0,1.0
Released,121.0,1.0,1.0,1.0,1.0,45.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,0.0,0.0,27.0,0.0,0.0,2.0,0.0,0.0
1,Executing,4.0,0.0,30.0,0.0,1.0,10.0,1.0,1.0
2,Released,121.0,1.0,1.0,1.0,1.0,45.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,0.0,0.0,27.0,0.0,0.0,2.0,0.0,0.0
1,Executing,4.0,0.0,30.0,0.0,1.0,10.0,1.0,1.0
2,Released,121.0,1.0,1.0,1.0,1.0,45.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()